# install deepdocdetect

In [1]:
!nvidia-smi

Sat Jun  1 11:11:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0              26W / 250W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!sudo apt-get -y -q install poppler-utils

# !pip install -q tensorpack

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libpoppler97 poppler-data
Suggested packages:
  ghostscript fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  libpoppler97 poppler-data poppler-utils
0 upgraded, 3 newly installed, 0 to remove and 65 not upgraded.
Need to get 2564 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 poppler-data all 0.4.9-2 [1475 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libpoppler97 amd64 0.86.1-0ubuntu1.4 [916 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.4 [174 kB]
Fetched 2564 kB in 1s (3604 kB/s)
Selecting previously unselected package poppler-data.
(Readi

In [3]:
#install dataflow
# !pip install -q --upgrade git+https://github.com/tensorpack/dataflow.git

In [4]:
!git clone https://github.com/deepdoctection/deepdoctection.git    
!cd deepdoctection && pip install -q ".[source-pt]"


#!pip install -q 'git+https://github.com/facebookresearch/detectron2.git'
# !pip install deepdoctection[pt]

Cloning into 'deepdoctection'...
remote: Enumerating objects: 12685, done.
remote: Counting objects: 100% (2113/2113), done.
remote: Compressing objects: 100% (589/589), done.
remote: Total 12685 (delta 1625), reused 1684 (delta 1524), pack-reused 10572
Receiving objects: 100% (12685/12685), 19.63 MiB | 24.37 MiB/s, done.
Resolving deltas: 100% (9726/9726), done.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.9.3 requires keras-core, which is not installed.
tensorflow-decis

# Imports

In [5]:
import os
os.environ['DD_USE_TORCH'] ="True"
os.environ['DD_USE_TF'] ="None"
os.environ['USE_TF'] ="None"


In [6]:
import deepdoctection as dd
from pathlib import Path
import cv2
import datetime
import tqdm
import gc
import pandas as pd
from glob import glob
from IPython.display import clear_output
import pickle

# import ray
# This automatically detects available resources in the single machine.
# ray.init()

2024-06-01 11:14:29.986066: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-01 11:14:29.986205: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-01 11:14:30.120462: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
[0601 11:14.38 @utils.py:161]  INF  NumExpr defaulting to 4 threads.
[0601 11:14.41 @file_utils.py:36]  INF  PyTorch version 2.1.2 available.
[0601 11:14.41 @file_utils.py:74]  INF  Disabling Tensorflow because USE_TORCH is set
[0601 11:14.41 @__init__.py:433]  WRN  Both DD_USE_TORCH and DD_USE_TF are set. Defaulting to PyTorch. If you want a different behaviour,

# Collect Papers for parsing

In [7]:
# find all the pdfs
papers_path = Path("/kaggle/input/astronomy-arxiv-papers/AstroPhysics_papers")

print("Number of papers in the dataset {}".format(len(list(papers_path.glob("*.pdf")))))

Number of papers in the dataset 50446


# custom logger

For logging the checking the parsing progress.

In [8]:
today = str(datetime.datetime.now()).rsplit(".",1)[0].split(" ")[0]


#define the logging paths
!mkdir ./logs/
logging_file_path = f"./logs/{today}_info.log"



def log(logging_file_path :str ,
        text:str):

    """Logger that logs the given text in defined logging file"""
    now = datetime.datetime.now()

    message = f"{now} --- {text}"

    with open(logging_file_path,"a") as file:
        file.write(message)

        return True

    return False


**Utils**

In [9]:
class CFG:
    max_chunk_tokens = 200
    logs_path = "/kaggle/working/logs/"
    papers_path = papers_path
    output_path = "/kaggle/working/parsed/"
    
    
!mkdir "./parsed/"

    
def delete_object(obj):
    del obj; gc.collect()
    return 

# Deep Doc Detect - Utils for document parsing


DeepDoctection is a python library which has pipelines for extracting document aware chunks. This is important as documents have a inherent structure within them, and with layout aware chunking, we can use this structure.

In [10]:

def init_analyzer(doc_lang='en'):
    """Initializes analyzer with a document language"""
    # analyzer
#     analyzer = dd.get_dd_analyzer(config_overwrite=[f"LANGUAGE={doc_lang}"])
    
    #init default
    analyzer = dd.get_dd_analyzer()

    return analyzer


def load_doc(path:str,analyzer):
    """Function to load a pdf file from the given path"""
    df = analyzer.analyze(path=path)
    df.reset_state()  # This method must be called just before starting the iteration. It is part of the API.

    doc=iter(df)

    return doc




def fetch_data_from_doc(doc):
    """Fetch the relevant doc data from the document"""

    continous_chunks = []
    page_wise_chunks = []
    tables = []

    try:
        for page in doc:

            chunks = get_page_data(page)

            continous_chunks.extend(chunks)

            page_wise_chunks.append(chunks)
            if len(page.tables)>0:
                tables.extend([t.html for t in page.tables])
    except Exception as e:
        log(logging_file_path =logging_file_path,
        text="fetch_data_from_doc_"+str(e))
        
    return (continous_chunks,page_wise_chunks,tables)



def get_page_data(page):

    all_blocks = []
    try:
        for block in page.chunks:
            block_data = get_block_data(block)
            all_blocks.append(block_data)

    except Exception as e:
        log(logging_file_path =logging_file_path  ,
        text="GET_PAGE_DATA - "+str(e))


    return all_blocks



def get_block_data(block):
    block_data = {'document_id': '',
           'image_id': '',
           'page_number': '',
           'annotation_id': '',
           'reading_order': '',
           'category_name': '',
           'text': ''}


    all_keys = list(block_data.keys())
    block_data_merged = {key:str(block[i]) for i,key in enumerate(all_keys)}

    block_data_merged['category_name']=block[-2].name


    return block_data_merged



def get_doc_data(filepath,analyzer):
    """Returns the parsed document data"""
    try:
        doc_loaded =load_doc(filepath,analyzer)

        continous_chunks, page_chunks,tables = fetch_data_from_doc(doc_loaded)

        doc_data = {"continous_chunks":continous_chunks,
                   "page_chunks":page_chunks,
                   "doc_id":continous_chunks[0].get("document_id",None),
                    "tables":tables,
                   "doc_path":filepath
                   }
        
        
        delete_object(doc_loaded)
        return doc_data
    except Exception as e:
        log(logging_file_path =logging_file_path  ,
        text="GB_"+str(e))

        return {}

**Chunking Logic** 

We will now define a logic to form chunks based on the category of the chunks

In [11]:
class Chunk:
    """class for a contigous chunk of text"""
    def __init__(self,
                 max_length = 200
                ):
        self.text = ""
        self.page_numbers = []
        self.categories = []
        self.max_length = max_length
        self.titles = []
        

    def accumulate(self,
                   block:dict)->None:

        page_num = block['page_number']
        category = block['category_name']
        text = block['text']


        if category =="title":
            self.titles.append(text)

        self.page_numbers.append(page_num)
        self.categories.append(category)




        if len(text) >=50:
            self.text += "\n" +  text

        # if not a continous passage of text
        else:
            self.text += 2 * "\n" +  text

        return True


    def chunking_rules(self,
                       block:dict):
        # the first element
        if len(self.categories) == 0:
            acc = self.accumulate(block)
            self.doc_id = block['document_id']

        # recent category is title
        elif ((len(self.text.split(" "))<self.max_length) and 'title' in self.categories[-2:]):
            acc = self.accumulate(block)


        #stop cases
        elif (len(self.text.split(" "))>self.max_length):
            acc = False

        # if we find another title
        elif ((len(self.text.split(" "))>(self.max_length // 4)) and (block['category_name']=='title')):
            acc = False


        # alternatively
        else:
            acc = self.accumulate(block)

        return acc



In [12]:
def layout_chunker(all_blocks:[{}])->[]:
    """function for parsing text into chunks"""
    all_chunks = []
    
    
    chunk = Chunk()
    
    for block in all_blocks:
        try:
            accumulated = chunk.chunking_rules(block)

            if not accumulated:
                all_chunks.append(chunk)
                chunk = Chunk()
                accumulated = chunk.chunking_rules(block)
                
        except Exception as e:
            log(logging_file_path =logging_file_path  ,text="layout_chunker_"+str(e))
            
    return all_chunks


**Class for saving the Document object.**

In [13]:
class Parsed_Doc:
    
    def __init__(self):
        pass
    
    def parse_chunks(self,loaded_doc:{}):
        self.doc_id = loaded_doc.get("doc_id","")
        self.doc_path = loaded_doc.get("doc_path","")
        self.chunks = layout_chunker(loaded_doc.get("continous_chunks",[{}]))
        self.tables = loaded_doc.get("tables",[])
        
        return True 
    

    
    

**Instantiate Deepdoc Analyzer instance**

The analyzer is the core of the pipeline,as it does the complete parsing work. We can also define a different langauge (out of the supported ones).

In [14]:
en_anaylzer = init_analyzer()
CFG.parsing_model = en_anaylzer

[0601 11:14.47 @dd.py:459]  INF  Config: 
 {'DEVICE': device(type='cuda'),
 'LANGUAGE': None,
 'LAYOUT_NMS_PAIRS': {'COMBINATIONS': None, 'PRIORITY': None, 'THRESHOLDS': None},
 'LIB': 'PT',
 'OCR': {'CONFIG': {'TESSERACT': 'dd/conf_tesseract.yaml'},
         'USE_DOCTR': False,
         'USE_TESSERACT': True,
         'USE_TEXTRACT': False,
         'WEIGHTS': {'DOCTR_RECOGNITION': {'PT': 'doctr/crnn_vgg16_bn/pt/crnn_vgg16_bn-9762b0b0.pt',
                                           'TF': 'doctr/crnn_vgg16_bn/tf/crnn_vgg16_bn-76b7f2c6.zip'},
                     'DOCTR_WORD': {'PT': 'doctr/db_resnet50/pt/db_resnet50-ac60cadc.pt',
                                    'TF': 'doctr/db_resnet50/tf/db_resnet50-adcafc63.zip'}}},
 'PDF_MINER': {'X_TOLERANCE': 3, 'Y_TOLERANCE': 3},
 'PT': {'CELL': {'FILTER': None,
                 'WEIGHTS': 'cell/d2_model_1849999_cell_inf_only.pt',
                 'WEIGHTS_TS': 'cell/d2_model_1849999_cell_inf_only.ts'},
        'ITEM': {'FILTER': None,
      

d2_model_0829999_layout_inf_only.pt:   0%|          | 0.00/275M [00:00<?, ?B/s]

Base-RCNN-FPN.yaml:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

CASCADE_RCNN_R_50_FPN_GN.yaml:   0%|          | 0.00/576 [00:00<?, ?B/s]

[0601 11:14.49 @config.py:66]  WRN  Loading config /root/.cache/deepdoctection/configs/dd/d2/layout/CASCADE_RCNN_R_50_FPN_GN.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.
[0601 11:14.51 @detection_checkpoint.py:38]  INF  [DetectionCheckpointer] Loading from /root/.cache/deepdoctection/weights/layout/d2_model_0829999_layout_inf_only.pt ...
[0601 11:14.51 @checkpoint.py:150]  INF  [Checkpointer] Loading from /root/.cache/deepdoctection/weights/layout/d2_model_0829999_layout_inf_only.pt ...


...

[0601 11:14.52 @config.py:66]  WRN  Loading config /root/.cache/deepdoctection/configs/dd/d2/item/CASCADE_RCNN_R_50_FPN_GN.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.
[0601 11:14.54 @detection_checkpoint.py:38]  INF  [DetectionCheckpointer] Loading from /root/.cache/deepdoctection/weights/item/d2_model_1639999_item_inf_only.pt ...
[0601 11:14.54 @checkpoint.py:150]  INF  [Checkpointer] Loading from /root/.cache/deepdoctection/weights/item/d2_model_1639999_item_inf_only.pt ...


d2_model_1849999_cell_inf_only.pt:   0%|          | 0.00/275M [00:00<?, ?B/s]

Base-RCNN-FPN.yaml:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

CASCADE_RCNN_R_50_FPN_GN.yaml:   0%|          | 0.00/699 [00:00<?, ?B/s]

[0601 11:14.56 @config.py:66]  WRN  Loading config /root/.cache/deepdoctection/configs/dd/d2/cell/CASCADE_RCNN_R_50_FPN_GN.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.
[0601 11:14.57 @detection_checkpoint.py:38]  INF  [DetectionCheckpointer] Loading from /root/.cache/deepdoctection/weights/cell/d2_model_1849999_cell_inf_only.pt ...
[0601 11:14.57 @checkpoint.py:150]  INF  [Checkpointer] Loading from /root/.cache/deepdoctection/weights/cell/d2_model_1849999_cell_inf_only.pt ...


# Parse docs into Text

**Sanity check**

Before we run our document parsing pipeline, we can do a sanity check of the method on a few examples to verify that it works as expected

In [15]:
check_fp = list(papers_path.glob("*.pdf"))[666]
loaded_doc = get_doc_data(check_fp,CFG.parsing_model)

loaded_doc.keys()

[0601 11:14.58 @doctectionpipe.py:84]  INF  Processing 0402338v3_0.pdf
/opt/conda/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
[0601 11:15.01 @context.py:126]  INF  ImageLayoutService total: 1.4484 sec.
[0601 11:15.01 @context.py:126]  INF  SubImageLayoutService total: 0.0 sec.
[0601 11:15.01 @context.py:126]  INF  SubImageLayoutService total: 0.0 sec.
[0601 11:15.01 @context.py:126]  INF  TableSegmentationService total: 0.0001 sec.
[0601 11:15.01 @context.py:126]  INF  TableSegmentationRefinementService total: 0.0 sec.
[0601 11:15.08 @context.py:126]  INF  TextExtractionService total: 6.7646 sec.
[0601 11:15.08 @context.py:126]  INF  MatchingService total: 0.0538 sec.
[0601 11:15.08 @context.py:126]  INF  TextOrde

dict_keys(['continous_chunks', 'page_chunks', 'doc_id', 'tables', 'doc_path'])

**Parsing the doc into chunk based on thier class and other chunking logic**

In [16]:
doc_ = Parsed_Doc()

doc_.parse_chunks(loaded_doc)

True

In [17]:
doc_.doc_id

'cb0a66bd-a55a-324a-a972-3d9401aeddfe'

In [18]:
#lets see how our algorithm is chunking the document

for ch in doc_.chunks:
    print(ch.text,end = "\n \n "+ "#"*50 +"\n")



Weighing the cusp at the Galactic Centre
N. Mouawap!, A. Ecxart', S. PraczNer!, R. Scuépet!, J. Mourraka!, and R. SPURZEM 2
| J. Physikalisches Institut, Universitat Kéln, Ziilpicher Str.77, 50937 Kéln, Germany zu 2 Astronomisches Recheninstitut, Ménchhofstr. 12-14, 69120 Heidelberg, Germany
Received <date>; accepted <date>; published online <date>
Abstract. As stars close to the galactic centre have short orbital periods it has been possible to trace large fractions of their orbits in the recent years. Previously the data of the orbit of the star S2 have been fitted with Keplerian orbits corresponding to a massive black hole (MBH) with a mass of Mgy=3-410°M, implying an insignificant cusp mass. However, it has also been shown that the central black hole resides in a ~1” diameter stellar cluster of a priori unknown mass. In a spherical potential which is neither Keplerian nor harmonic, orbits will precess resulting in inclined rosetta shaped trajectories on the sky. In this case, the

In [19]:
from IPython.core.display import HTML

HTML(doc_.tables[0])

,"R,[arcsec]",Ri[pc],Mi[Mo],ap; [Mo],pi(0)
,,0.015,8540,1080,6.010
,13,0.5,1.8x10®,10,3.5x10®
,52,2.0,2.7x10®,0.25,7.9x10*
,97,3.8,6.7x10°,0.01,2.9x10*
,220,8.5,13.1x10°,0.002,5.1«10?
,321,12.4,27.6x10°,0.001,3.4x10?


In [20]:
def pickle_obj(obj,path):
    with open(path,"wb") as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
    return 


In [21]:
pickle_obj(doc_,CFG.output_path + "check.pkl")

# Parsing all the docs 

In [22]:
# files to store parsing logs or errors 
progress_log = "./parsing_progress.log"
parse_fail_log = "./parse_fail_log.log"
parsed_logs = "./files_processed.log"




In [23]:
def process_document(filepath):
    file_path_str = str(filepath)
    file_name = file_path_str.split("/")[-1].split(".")[0]
    try:
        # load the doc in deepdoc detect
        loaded_doc = get_doc_data(file_path_str,CFG.parsing_model)

        #chunk and save as obj
        parsed_doc = Parsed_Doc()
        parsed_doc.parse_chunks(loaded_doc)
        
        #save doc object 
        pickle_obj(parsed_doc,CFG.output_path + f"{file_name}.pkl")
        
        #delete objs
        delete_object(loaded_doc)
        delete_object(parsed_doc)
    
    except Exception as e:
        log(parse_fail_log,f"Processing failed for ##{file_path_str}## \n {e}")
        
    return file_path_str

In [24]:
for idx,filepath in tqdm.tqdm(enumerate(papers_path.glob("*.pdf"))):
    
    if (idx%500==0) or (idx==0):
        log(progress_log,f"Processed {idx} files.")
        
    processed = process_document(filepath)
    log(parsed_logs,f"Processed {idx} files.")
    
    clear_output(wait=True)

194it [7:25:48, 108.92s/it][0601 18:43.19 @pdf_utils.py:70]  INF  qpdf is not installed. If the document must be decrypted please ensure that it is installed
[0601 18:43.19 @pdf_utils.py:103]  ERR  pdf document /kaggle/input/astronomy-arxiv-papers/AstroPhysics_papers/0108380v1.pdf cannot be decrypted and therefore cannot be processed further.
194it [7:25:48, 137.88s/it]


SystemExit: 

/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# Whats Next 

* Extraction of more features, summary etc from the documents
* we will explore how to store these chunks in a vector database in the next notebook